In [1]:
import xarray as xr
import xpublish
import zarr

import numpy as np
from json import loads

from fsspec.implementations.http import HTTPFileSystem

from xarray.testing import assert_identical, assert_equal, assert_chunks_equal

In [2]:
from dask.distributed import Client

In [3]:
client = Client()
client

/Users/jhamman/miniconda3/lib/python3.6/site-packages/distributed/dashboard/core.py:79: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


Client Scheduler: tcp://127.0.0.1:59701 Dashboard: http://127.0.0.1:59702/status,Cluster Workers: 4 Cores: 4 Memory: 34.36 GB


In [4]:
fs = HTTPFileSystem()
http_map = fs.get_mapper('http://0.0.0.0:9000')

In [5]:
for key in ['.zmetadata', 'keys']:
    print(key, http_map[key], '\n')

.zmetadata b'{"zarr_consolidated_format":1,"metadata":{".zgroup":{"zarr_format":2},".zattrs":{"Conventions":"COARDS","title":"4x daily NMC reanalysis (1948)","description":"Data is from NMC initialized reanalysis\\n(4x/day).  These are the 0.9950 sigma level values.","platform":"Model","references":"http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanalysis.html"},"lat/.zattrs":{"standard_name":"latitude","long_name":"Latitude","units":"degrees_north","axis":"Y","_ARRAY_DIMENSIONS":["lat"]},"lat/.zarray":{"compressor":{"id":"blosc","cname":"lz4","clevel":5,"shuffle":1,"blocksize":0},"filters":null,"chunks":[25],"dtype":"<f4","fill_value":"NaN","order":"C","shape":[25],"zarr_format":2},"air/.zattrs":{"long_name":"4xDaily Air temperature at sigma level 995","units":"degK","precision":2,"GRIB_id":11,"GRIB_name":"TMP","var_desc":"Air temperature","dataset":"NMC Reanalysis","level_desc":"Surface","statistic":"Individual Obs","parent_stat":"Other","actual_range":[185.16000366210938,322.1

In [6]:
d = loads(http_map['.zmetadata'])
d['metadata']['air/.zattrs']

{'long_name': '4xDaily Air temperature at sigma level 995',
 'units': 'degK',
 'precision': 2,
 'GRIB_id': 11,
 'GRIB_name': 'TMP',
 'var_desc': 'Air temperature',
 'dataset': 'NMC Reanalysis',
 'level_desc': 'Surface',
 'statistic': 'Individual Obs',
 'parent_stat': 'Other',
 'actual_range': [185.16000366210938, 322.1000061035156],
 'scale_factor': 0.01,
 '_ARRAY_DIMENSIONS': ['time', 'lat', 'lon']}

In [7]:
zg = zarr.open_consolidated(http_map, mode='r')

In [8]:
zg.tree()

ModuleNotFoundError: No module named 'ipytree'

/
 ├── air (2920, 25, 53) int16
 ├── lat (25,) float32
 ├── lon (53,) float32
 └── time (2920,) float32

In [9]:
ds = xr.open_zarr(http_map, consolidated=True, decode_cf=True)
ds

<xarray.Dataset>
Dimensions:  (lat: 25, lon: 53, time: 2920)
Coordinates:
  * lat      (lat) float32 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
  * lon      (lon) float32 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
  * time     (time) datetime64[ns] 2013-01-01 ... 2014-12-31T18:00:00
Data variables:
    air      (time, lat, lon) float32 dask.array<chunksize=(2920, 5, 5), meta=np.ndarray>
Attributes:
    Conventions:  COARDS
    title:        4x daily NMC reanalysis (1948)
    description:  Data is from NMC initialized reanalysis\n(4x/day).  These a...
    platform:     Model
    references:   http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanaly...

In [10]:
ds_tutorial = xr.tutorial.open_dataset(
    'air_temperature',
    chunks=dict(lat=5, lon=5),
    decode_cf=True)

In [11]:
ds_tutorial.air.attrs

{'long_name': '4xDaily Air temperature at sigma level 995',
 'units': 'degK',
 'precision': 2,
 'GRIB_id': 11,
 'GRIB_name': 'TMP',
 'var_desc': 'Air temperature',
 'dataset': 'NMC Reanalysis',
 'level_desc': 'Surface',
 'statistic': 'Individual Obs',
 'parent_stat': 'Other',
 'actual_range': array([185.16, 322.1 ], dtype=float32)}

In [12]:
def test(actual, expected, index):
    '''a simple equality test with index as a parameter'''
    assert np.array_equal(actual[index].values, expected[index].values)

In [13]:
# test a bunch of indexing patterns
for index in [(0, 0, 0),
              (slice(0, 4), 0, 0),
              (slice(0, 4), slice(0, 4), 0),
              (slice(0, 4), slice(0, 4), slice(0, 4)),
              (slice(-4), slice(0, 4), slice(0, 4)),
              (slice(None), slice(0, 4), slice(0, 4)),
              (slice(None), slice(None), slice(0, 4)),
              (slice(None), slice(None), slice(None)),
    ]:
    print(index)
    test(ds_tutorial['air'], ds['air'], index)

(0, 0, 0)
(slice(0, 4, None), 0, 0)
(slice(0, 4, None), slice(0, 4, None), 0)
(slice(0, 4, None), slice(0, 4, None), slice(0, 4, None))
(slice(None, -4, None), slice(0, 4, None), slice(0, 4, None))
(slice(None, None, None), slice(0, 4, None), slice(0, 4, None))
(slice(None, None, None), slice(None, None, None), slice(0, 4, None))
(slice(None, None, None), slice(None, None, None), slice(None, None, None))


In [17]:
assert_equal(ds, ds_tutorial)
assert_chunks_equal(ds, ds_tutorial)
assert_identical(ds, ds_tutorial)